In [11]:
import os
import re
import sys
import json
import random
import itertools
from copy import copy
import numpy as np
import pandas as pd
from nltk import ngrams
from pandas.io.json import json_normalize
import keras
from keras.layers import LSTM, Dense, Conv1D, MaxPool1D, AveragePooling1D, Flatten
from keras.models import Sequential, load_model
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

## Utils

In [12]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, sentences, next_words, maxlen, word_index, batch_size=32, shuffle=True):
        self.batch_size = batch_size
        self.next_words = next_words
        self.sentences = sentences
        self.shuffle = shuffle
        self.maxlen = maxlen
        self.word_index = word_index
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.sentences) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        sentences_temp = [self.sentences[k] for k in indexes]
        next_words_temp = [self.next_words[l] for l in indexes]

        # Generate data
        X, y = self.__data_generation(sentences_temp, next_words_temp)

        return X, y
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.sentences))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, sentences, next_words):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((len(sentences), self.maxlen, len(word_index))) # (sentences)
        y = np.zeros((len(sentences), len(word_index)), dtype=np.bool)

        # Generate data
        for i, sentence in enumerate(sentences):
            for t, word in enumerate(sentence):
                X[i, t, word_index[word]] = 1    # one hot encoding
                y[i, word_index[next_words[i]]] = 1

        return X, y

def load_json(json_path, artists=[]):
    if (os.path.isfile(json_path)):
        print("json")
        with open(json_path) as f:
            song_data = json.load(f)
            return song_data['songs']
        
    elif (os.path.isdir(json_path)):
        data = []
        json_files = []
        if (len(artists) > 0):
            for artist in artists:
                json_files = json_files + [json_file for json_file in os.listdir(json_path) if ((json_file.endswith('.json')) & (artist in json_file))]
        else:
            json_files = [json_file for json_file in os.listdir(json_path) if json_file.endswith('.json')]

        for json_file in json_files:
            path_to_json = os.path.join(json_path, json_file)
            with open(path_to_json) as f:
                song_data = json.load(f)
                data = data + song_data['songs']
        
        return data
    
    
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    
    return distribution / np.sum(distribution)


def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

def normalize_lyric(text, lower=True):
    if lower:
        text = text.lower()
    text = re.sub('\[.+\](\\n)|\[.+\](\(.*\))', '', text)
    return text 

## Load Data

In [13]:
json_path = '../data/deutsch'
artists = ['Bushido']

data = load_json(json_path, artists)
df = json_normalize(data)
lyrics = df.lyrics.map(lambda lyric: normalize_lyric(lyric))

## Preprocess Data

In [14]:
lyrics_in_words = []
for lyric in lyrics:
    lyric = lyric.replace('\n', ' \n ').lower()
    words = lyric.split(' ')
    lyrics_in_words.append(words)
    
print('Corpus length in words:', len(list(itertools.chain(*lyrics_in_words))))

Corpus length in words: 58768


In [15]:
words = set(list(itertools.chain(*lyrics_in_words)))
print('Unique words:', len(words))
word_index = dict((c, i) for i, c in enumerate(words))
index_word = dict((i, c) for i, c in enumerate(words))

Unique words: 9013


In [16]:
maxlen = 7
step = 1

sentences = []
next_words = []
for lyric in lyrics_in_words:
    for i in range(0, len(lyric) - maxlen, step): # iterates by step size
        sentences.append(lyric[i: i + maxlen]) # get maxlen amount of characters
        next_words.append(lyric[i + maxlen])

## Create Datagenerators

In [17]:
sentences_train, sentences_test, next_words_train, next_words_test = train_test_split(sentences, next_words)

In [18]:
EPOCHS = 1
BATCH_SIZE = 64
DIR = '../outputs/wordbased/CNN_Simple_WordBased_{}_E{}_BS{}_ML{}_SS{}'.format(artists[0], EPOCHS, BATCH_SIZE, maxlen, step)

if not os.path.exists(DIR):
    os.makedirs(DIR)

In [19]:
training_generator = DataGenerator(sentences_train, next_words_train, maxlen, word_index, batch_size=BATCH_SIZE)
test_generator = DataGenerator(sentences_test, next_words_test, maxlen, word_index, batch_size=BATCH_SIZE)

## Callbacks

In [20]:
tensorboard = TensorBoard(log_dir=os.path.join(DIR, 'logs'), write_images=True, write_grads=True)
modelCheckpoint_best = ModelCheckpoint(filepath=os.path.join(DIR, "model_best.h5"), save_best_only=True)
modelCheckpoint = ModelCheckpoint(filepath=os.path.join(DIR, "model.h5"), save_best_only=False)
#earlyStopping = EarlyStopping(patience=8)

## Build Model

In [23]:
model = Sequential()
model.add(Conv1D(input_shape=(maxlen, len(words)),
                filters=32,
                kernel_size=7,
                padding='same',
                activation='relu',
                strides=1))
model.add(MaxPool1D(pool_size=2))

model.add(Conv1D(filters=64,
                kernel_size=3,
                padding='same',
                activation='relu',
                strides=1))
#model.add(MaxPool1D(pool_size=2))

"""model.add(Conv1D(filters=64,
                kernel_size=3,
                padding='same',
                activation='relu',
                strides=1))
model.add(MaxPool1D(pool_size=2))

model.add(Conv1D(filters=32,
                kernel_size=3,
                padding='same',
                activation='relu',
                strides=1))"""
model.add(AveragePooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(len(words), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Train Model

In [24]:
model.fit_generator(training_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=[tensorboard, modelCheckpoint, modelCheckpoint_best])

Epoch 1/1
680/680 [==============================] - 79s 116ms/step - loss: 6.7232 - val_loss: 6.5135


## Generate Text

In [25]:
temperature = 0.5
random.seed(3004)

lyrics_index = random.randint(0, len(lyrics))
chosen_lyric = lyrics_in_words[lyrics_index]
start_index = random.randint(0, len(chosen_lyric) - maxlen - 1)
generated_text_temp = chosen_lyric[start_index: start_index + maxlen]
generated_text = copy(generated_text_temp)
print(" ".join(generated_text) + '_')
#print('\n___________________\n')
for i in range(100):
    sampled = np.zeros((1, maxlen, len(words)))
            
    for t, word in enumerate(generated_text_temp):
        sampled[0, t, word_index[word]] = 1.
                      
    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_word = index_word[next_index]
    generated_text_temp.append(next_word)
    generated_text.append(next_word)
    generated_text_temp = generated_text_temp[1:]
    sys.stdout.write(" " + next_word)

anderen bushido geben 
  
 ich_
 ich der 
 
 
 du hier ein der auch 
 ich 
 der kein 
 
 und du die 
 
 die wie 
 ich 
 wer 
 
 ich bin wieder du 
 
 ich du als 
 
 nicht ich der dir 
 
 ich nie 
 ich 
 
 sie wie nicht 
 ich kannst du du 
 ich 
 ich du 
 ich der ein hab 
 ist 
 ich der 
 
 ich ich deinen du die warst 
 ich 
 du die 
 sonny der 
 wie ist 
 ich es gibt 
